In [243]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import Imputer
pd.options.mode.chained_assignment = None

# Data Cleaning
missing: remove blanks
data conversion

In [244]:
def correctcol(data):
    new_header = data.iloc[0] 
    data = data[1:] 
    data.columns = new_header
    data = data.loc[:, data.columns.notnull()]
    return data
def filtercustid(data):
    final = data.loc[data['customer_id'] <= 3500]
    return final
def cleantrans(data):
    data = correctcol(data)
    data = filtercustid(data)
    data = data.loc[data['order_status'] != 'Cancelled']
    data['profit'] = data['list_price'] - data['standard_cost']
    data['online_order'].replace(np.nan, 'unknown',inplace=True)
    data['brand'].replace('', np.nan, inplace=True)
    data.dropna(subset=['brand'], inplace=True)
    return data
def cleancustadd(data):
    data = correctcol(data)
    data["state"].replace({"New South Wales": "NSW", "Victoria": "VIC"}, inplace=True)
    return data
def cleancustdemo(data):
    data = correctcol(data)
    data = filtercustid(data)
    a = data.loc[data['deceased_indicator'] != 'Y']
    a.replace({"Male": "M", "Female": "F", "Femal":"F"}, inplace=True)
    a['DOB'] = pd.to_datetime(a['DOB'], errors='coerce')
    a['age']= (now.year - a['DOB'].dt.year) - ((now.month - a['DOB'].dt.month) < 0)
    a['job_title'].replace(np.nan, 'unknown',inplace=True)
    a['job_industry_category'].replace(np.nan, 'unknown',inplace=True)
    a['tenure'].replace('', np.nan, inplace=True)
    a.dropna(subset=['tenure'], inplace=True)
    del a['DOB']
    del a['default']
    return a

In [245]:
trans = pd.read_excel('KPMG.xlsx','Transactions')
custadd = pd.read_excel('KPMG.xlsx', 'CustomerAddress')
custdemo= pd.read_excel('KPMG.xlsx', 'CustomerDemographic')

In [246]:
custdemo = cleancustdemo(custdemo)
custadd = cleancustadd(custadd)
trans =  cleantrans(trans)


In [247]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

#make a lists, indicating which features
# will be imputed with each method

features_numeric = ['age']

# then instantiate the imputers, within a pipeline
# we create one imputer for numerical and one imputer
# for categorical

# this imputer imputes with the mean
imputer_numeric = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])



# then put the features list and the transformers together
# using the column transformer

preprocessor = ColumnTransformer(transformers=[('imputer_numeric',
                                                imputer_numeric,
                                                features_numeric)])

# now fit the preprocessor
preprocessor.fit(custdemo[['age']])

# and now can impute the data
# remember it returs a numpy array

custdemo['age'] = preprocessor.transform(custdemo[['age']]).ravel()
# X_test = preprocessor.transform(X_test)

#  Data Exploration

Transaction 